# 1. Preprocess-Stage1-Group-All-TrainData-KFold-Test

## Run name

In [1]:
import time
project_name = 'Google-LandMark-Rec2019'
step_name = '1-Preprocess-Stage1-Group-All-TrainData-KFold-Test'
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: Google-LandMark-Rec2019_1-Preprocess-Stage1-Group-All-TrainData-KFold-Test_20190510-152539


## Important params

In [2]:
import multiprocessing
import random

cpu_amount = multiprocessing.cpu_count()
random_num = random.randint(0, 10000)
# train_tar_count = 500
train_data_ratio = 0.95

# print('train_tar_count:', train_tar_count)
print('cpu_amount: ', cpu_amount)
print('random_num:', random_num)
print(train_data_ratio)

cpu_amount:  4
random_num: 1292
0.95


## Import PKGs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display
from sklearn.model_selection import train_test_split, KFold

import os, sys, gc, math, shutil, zipfile, pickle, h5py, re, tarfile
import urllib, xlsxwriter
from tqdm import tqdm
import hashlib

## Project folders

In [4]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

md5sum_folder = os.path.join(input_folder, 'md5sum')
if not os.path.exists(md5sum_folder):
    os.mkdir(md5sum_folder)
    print('create folder:', md5sum_folder)
    
train_tar_folder = os.path.join(input_folder, 'train_tar')
if not os.path.exists(train_tar_folder):
    os.mkdir(train_tar_folder)
    print('create folder:', train_tar_folder)
    
train_untar_folder = os.path.join(input_folder, 'train_untar')
if not os.path.exists(train_untar_folder):
    os.mkdir(train_untar_folder)
    print('create folder:', train_untar_folder)
    
all_image_folder = os.path.join(input_folder, 'all_image')
if not os.path.exists(all_image_folder):
    os.mkdir(all_image_folder)
    print('create folder:', all_image_folder)
    
test_image_folder = os.path.join(input_folder, 'test_image')
# if not os.path.exists(test_image_folder):
#     os.mkdir(test_image_folder)
#     print('create folder:', test_image_folder)

data_test_folder = os.path.join(input_folder, 'data_test')
if not os.path.exists(data_test_folder):
    os.mkdir(data_test_folder)
    print('create folder:', data_test_folder)
    
data_test_subfolder = os.path.join(data_test_folder, 'test')
if not os.path.exists(data_test_subfolder):
    os.mkdir(data_test_subfolder)
    print('create folder:', data_test_subfolder)
else:
    print('folder exists:', data_test_subfolder)
# data_train_folder = os.path.join(input_folder, 'data_train')
# if not os.path.exists(data_train_folder):
#     os.mkdir(data_train_folder)
#     print('create folder:', data_train_folder)
    
# data_val_folder = os.path.join(input_folder, 'data_val')
# if not os.path.exists(data_val_folder):
#     os.mkdir(data_val_folder)
#     print('create folder:', data_val_folder)

create folder: /data/landmark-recognition-2019/input/data_test/test


In [5]:
!pwd
!ls ./input -hl

/data/landmark-recognition-2019
total 1.8G
drwxr-xr-x     2 root   root   4.0K May  4 08:28 50000_00
drwxr-xr-x     2 root   root   4.0K May  4 08:28 50000_01
drwxr-xr-x     2 root   root   4.0K May  4 08:28 50000_02
drwxr-xr-x     2 root   root   4.0K May  4 08:28 50000_03
drwxr-xr-x     2 root   root   4.0K May  4 08:28 50000_04
drwxr-xr-x     2 root   root   160M May  2 21:14 all_image
drwxr-xr-x     3 root   root   4.0K May 10 15:25 data_test
drwxr-xr-x 34010 root   root   824K May  4 09:55 data_train_00
drwxr-xr-x 34216 root   root   772K May  4 10:07 data_train_01
drwxr-xr-x 34205 root   root   836K May  4 10:18 data_train_02
drwxr-xr-x 34053 root   root   836K May  4 10:30 data_train_03
drwxr-xr-x 34183 root   root   812K May  4 10:42 data_train_04
drwxr-xr-x 34082 root   root   824K May  4 10:53 data_train_05
drwxr-xr-x 33987 root   root   804K May  4 11:03 data_train_06
drwxr-xr-x 34110 root   root   832K May  4 11:13 data_train_07
drwxr-xr-x 34196 root   root   804K May  4 11

In [6]:
!ls ./input/all_image/ -U|head -5

e42936e3627d8989.jpg
96d0f4fec18d8c84.jpg
02fd1c4af95765fd.jpg
37bdd02aebba82a6.jpg
b732241b0be6a043.jpg
ls: write error: Broken pipe


In [7]:
# !wget https://s3.amazonaws.com/google-landmark/metadata/train.csv
# !wget https://s3.amazonaws.com/google-landmark/metadata/train_attribution.csv

In [8]:
train_csv = os.path.join(input_folder, 'train.csv')
test_csv = os.path.join(input_folder, 'test.csv')
boxes_split1_csv = os.path.join(input_folder, 'google_landmark_boxes_split1.csv')
boxes_split2_csv = os.path.join(input_folder, 'google_landmark_boxes_split2.csv')
recognition_sample_submission_csv = os.path.join(input_folder, 'recognition_sample_submission.csv')

print(train_csv)
print(test_csv)
print(boxes_split1_csv)
print(boxes_split2_csv)
print(recognition_sample_submission_csv)


/data/landmark-recognition-2019/input/train.csv
/data/landmark-recognition-2019/input/test.csv
/data/landmark-recognition-2019/input/google_landmark_boxes_split1.csv
/data/landmark-recognition-2019/input/google_landmark_boxes_split2.csv
/data/landmark-recognition-2019/input/recognition_sample_submission.csv


In [9]:
# def split_group(df, all_image_folder, data_set_name='demo', split_size=50000):
#     split_amount = math.ceil(df.shape[0]/split_size)
#     print('split_amount:', split_amount)
        
#     data_item_invalid_count = 0
#     data_item_softlink_count = 0
#     for i, start_index in enumerate(range(0, df.shape[0], split_size)):
#         print(i, start_index, end='\t')
#         split_df = df.loc[df.index[start_index: (start_index+split_size)]]
#         print(split_df.shape, end='\t')

#         data_item_folder = os.path.join(input_folder, '%s_%02d' % (data_set_name, i))
#         if not os.path.exists(data_item_folder):
#             os.mkdir(data_item_folder)
#             print('Folder crated:', data_item_folder, end='\t')
#         for idx, row in split_df.iterrows():
#             src_image_path = os.path.join(all_image_folder, '%s.jpg' % row['id'])
#         #     print(src_image_path)
#             if not os.path.exists(src_image_path):
#                 data_item_invalid_count += 1
#                 continue
#         #     print(src_image_path)
#             target_image_folder = os.path.join(data_item_folder, '%06d' % row['landmark_id'])
#         #     print(target_image_folder)
#             if not os.path.exists(target_image_folder):
#                 os.mkdir(target_image_folder)
#         #         print('Folder created:', target_image_folder)
#             target_image_path = os.path.join(target_image_folder, '%s.jpg' % row['id'])
#             if not os.path.exists(target_image_path):
#                 data_item_softlink_count += 1
#                 os.symlink(src_image_path, target_image_path)
#             if data_item_softlink_count % 1000 == 0:
#                 print('-', end='')
# #             if data_item_softlink_count % 10000 == 0:
# #                 print('|')

#         print('data_item_invalid_count:', data_item_invalid_count, end='\t')
#         print('data_item_softlink_count:', data_item_softlink_count)
#         print('*' * 40)
#     print('*' * 80)


In [10]:
images = os.listdir(test_image_folder)
count = 0
for file_name in images:
    src_image_path = os.path.join(test_image_folder, file_name)
    target_image_path = os.path.join(data_test_subfolder, file_name)
#     print(src_image_path, target_image_path)
    if not os.path.exists(target_image_path):
        os.symlink(src_image_path, target_image_path)
    count += 1
    if count % 100 == 0:
        print('*', end='')
    if count % 10000 == 0:
        print('|')
        
print(count)

****************************************************************************************************|
****************************************************************************************************|
****************************************************************************************************|
****************************************************************************************************|
****************************************************************************************************|
****************************************************************************************************|
****************************************************************************************************|
****************************************************************************************************|
****************************************************************************************************|
**********************************************************************************

In [11]:
print('Time elapsed: %.1fs' % (time.time() - t0))
print(run_name)

Time elapsed: 14.1s
Google-LandMark-Rec2019_1-Preprocess-Stage1-Group-All-TrainData-KFold-Test_20190510-152539
